In [ ]:
# The goal of this homework is to create a regression model for predicting the 
# car fuel efficiency (column 'fuel_efficiency_mpg').

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
## Read in the csv

df = pd.read_csv('car_fuel_efficiency.csv')
df

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369
...,...,...,...,...,...,...,...,...,...,...,...
9699,140,5.0,164.0,2981.107371,17.3,2013,Europe,Diesel,Front-wheel drive,NaN,15.101802
9700,180,NaN,154.0,2439.525729,15.0,2004,USA,Gasoline,All-wheel drive,0.0,17.962326
9701,220,2.0,138.0,2583.471318,15.1,2008,USA,Diesel,All-wheel drive,-1.0,17.186587
9702,230,4.0,177.0,2905.527390,19.4,2011,USA,Diesel,Front-wheel drive,1.0,15.331551


In [5]:
new_df = df[['engine_displacement','horsepower','vehicle_weight', 'model_year', 'fuel_efficiency_mpg']]

In [7]:
new_df

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369
...,...,...,...,...,...
9699,140,164.0,2981.107371,2013,15.101802
9700,180,154.0,2439.525729,2004,17.962326
9701,220,138.0,2583.471318,2008,17.186587
9702,230,177.0,2905.527390,2011,15.331551


In [9]:
### Question 1 ###
# There's one column with missing values. What is it?

new_df.isnull().sum() # horsepower

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [11]:
# new_df.dtypes[new_df.dtypes == 'object'].index

In [13]:
new_df.head

<bound method NDFrame.head of       engine_displacement  horsepower  vehicle_weight  model_year  \
0                     170       159.0     3413.433759        2003   
1                     130        97.0     3149.664934        2007   
2                     170        78.0     3079.038997        2018   
3                     220         NaN     2542.392402        2009   
4                     210       140.0     3460.870990        2009   
...                   ...         ...             ...         ...   
9699                  140       164.0     2981.107371        2013   
9700                  180       154.0     2439.525729        2004   
9701                  220       138.0     2583.471318        2008   
9702                  230       177.0     2905.527390        2011   
9703                  270       140.0     2908.043477        2005   

      fuel_efficiency_mpg  
0               13.231729  
1               13.688217  
2               14.246341  
3               16.912736  
4

In [26]:
### Question 2 ###
# What's the median (50% percentile) for variable 'horsepower'?
new_df.describe() # 149.000000

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
count,9704.000000,8996.000000,9704.000000,9704.000000,9704.000000
mean,199.708368,149.657292,3001.280993,2011.484027,14.985243
std,49.455319,29.879555,497.894860,6.659808,2.556468
min,10.000000,37.000000,952.681761,2000.000000,6.200971
25%,170.000000,130.000000,2666.248985,2006.000000,13.267459
50%,200.000000,149.000000,2993.226296,2012.000000,15.006037
75%,230.000000,170.000000,3334.957039,2017.000000,16.707965
max,380.000000,271.000000,4739.077089,2023.000000,25.967222


In [28]:
### Prepare and split the dataset ###
# Shuffle the dataset (the filtered one you created above), use seed 42.
# Split your data in train/val/test sets, with 60%/20%/20% distribution.
# Use the same code as in the lectures

np.random.seed(42)

n = len(new_df)

n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

new_df_shuffled = new_df.iloc[idx]

new_df_train = new_df_shuffled.iloc[:n_train].copy()
new_df_train2 = new_df_train
new_df_val = new_df_shuffled.iloc[n_train:n_train+n_val].copy()
new_df_test = new_df_shuffled.iloc[n_train+n_val:].copy()

In [30]:
# Question 3 - part 1 - fill horsepower with 0
# We need to deal with missing values for the column from Q1.
# We have two options: fill it with 0 or with the mean of this variable.
# Try both options. For each, train a linear regression model 
# without regularization using the code from the lessons.
# --
new_df_train['horsepower'].fillna(0)

# For computing the mean, use the training only!
# Use the validation dataset to evaluate the models and compare the RMSE of each option.
# Round the RMSE scores to 2 decimal digits using round(score, 2)
# Which option gives better RMSE?

483     144.0
7506    141.0
8795    155.0
1688    206.0
6217    111.0
        ...  
1696    139.0
5685    132.0
6735    152.0
2052    154.0
1776    152.0
Name: horsepower, Length: 5824, dtype: float64

In [32]:
new_df_train.describe()  # 138.529190

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
count,5824.000000,5395.000000,5824.000000,5824.000000,5824.000000
mean,199.429945,149.544764,3006.672919,2011.444712,14.954499
std,49.543177,29.610498,497.856998,6.669193,2.558077
min,10.000000,46.000000,952.681761,2000.000000,6.846256
25%,170.000000,129.000000,2669.313374,2006.000000,13.239002
50%,200.000000,149.000000,2993.348283,2011.000000,14.986637
75%,230.000000,170.000000,3342.151520,2017.000000,16.681672
max,380.000000,246.000000,4610.973229,2023.000000,25.967222


In [34]:
new_df_train.head(25)

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
483,220,144.0,2535.887591,2009,16.642943
7506,160,141.0,2741.170484,2019,16.298377
8795,230,155.0,2471.880237,2017,18.591822
1688,150,206.0,3748.164469,2015,11.818843
6217,300,111.0,2135.716359,2006,19.402209
4562,180,120.0,3551.398040,2013,12.145734
5885,210,118.0,3620.351977,2018,11.378399
3746,150,152.0,2524.407794,2012,16.667294
7109,160,151.0,2752.964230,2009,15.819367
2698,250,147.0,3202.856567,2020,14.100998


In [28]:
new_df_train.describe() # 138.529190

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
count,5824.000000,5824.000000,5824.000000,5824.000000,5824.000000
mean,199.429945,138.529190,3006.672919,2011.444712,14.954499
std,49.543177,48.357282,497.856998,6.669193,2.558077
min,10.000000,0.000000,952.681761,2000.000000,6.846256
25%,170.000000,123.000000,2669.313374,2006.000000,13.239002
50%,200.000000,146.000000,2993.348283,2011.000000,14.986637
75%,230.000000,168.000000,3342.151520,2017.000000,16.681672
max,380.000000,246.000000,4610.973229,2023.000000,25.967222


In [36]:
new_df_train.isnull().sum()

engine_displacement      0
horsepower             429
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [38]:
### Question 3 - part 2 - fill horsepower with the mean ###

new_df_train2.fillna(138.529190)

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
483,220,144.0,2535.887591,2009,16.642943
7506,160,141.0,2741.170484,2019,16.298377
8795,230,155.0,2471.880237,2017,18.591822
1688,150,206.0,3748.164469,2015,11.818843
6217,300,111.0,2135.716359,2006,19.402209
...,...,...,...,...,...
1696,260,139.0,2606.972984,2009,16.964054
5685,280,132.0,4004.214323,2014,10.059094
6735,210,152.0,2500.175687,2020,17.593237
2052,250,154.0,2254.313245,2002,18.925748


In [40]:
new_df_train2.head(25)

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
483,220,144.0,2535.887591,2009,16.642943
7506,160,141.0,2741.170484,2019,16.298377
8795,230,155.0,2471.880237,2017,18.591822
1688,150,206.0,3748.164469,2015,11.818843
6217,300,111.0,2135.716359,2006,19.402209
4562,180,120.0,3551.398040,2013,12.145734
5885,210,118.0,3620.351977,2018,11.378399
3746,150,152.0,2524.407794,2012,16.667294
7109,160,151.0,2752.964230,2009,15.819367
2698,250,147.0,3202.856567,2020,14.100998


In [42]:
new_df_train2.isnull().sum()

engine_displacement      0
horsepower             429
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [44]:
new_df_train2.describe()  # 149.544764

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
count,5824.000000,5395.000000,5824.000000,5824.000000,5824.000000
mean,199.429945,149.544764,3006.672919,2011.444712,14.954499
std,49.543177,29.610498,497.856998,6.669193,2.558077
min,10.000000,46.000000,952.681761,2000.000000,6.846256
25%,170.000000,129.000000,2669.313374,2006.000000,13.239002
50%,200.000000,149.000000,2993.348283,2011.000000,14.986637
75%,230.000000,170.000000,3342.151520,2017.000000,16.681672
max,380.000000,246.000000,4610.973229,2023.000000,25.967222


In [ ]:
# Use the validation dataset to evaluate the models - horsepower with 0
def prepare_X(new_df):
    new_df = df.copy()
    features = base.copy()

    df['age'] = 2017 - df.year
    features.append('age')

    for v in [2, 3, 4]:
        feature = 'num_doors_%s' % v
        df[feature] = (df['number_of_doors'] == v).astype(int)
        features.append(feature)

    for v in ['chevrolet', 'ford', 'volkswagen', 'toyota', 'dodge']:
        feature = 'is_make_%s' % v
        df[feature] = (df['make'] == v).astype(int)
        features.append(feature)

    df_num = df[features]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [ ]:
# Use the validation dataset to evaluate the models - horsepower with the mean